In [10]:
from pathlib import Path

paths = list(Path("./PSsample/").glob("*.wav"))
import pandas as pd
pd.DataFrame(data={"path": [str(p) for p in paths],
                   "target": ["sadness" for p in paths]}
             ).to_csv("014_PSs.csv")
    
    
checkpoints = [
    # "facebook/wav2vec2-large-slavic-voxpopuli-v2",
    # "facebook/wav2vec2-large-960h-lv60-self",
    "classla/wav2vec2-large-slavic-parlaspeech-hr",
]
optimal_epochs = {
    "facebook/wav2vec2-large-960h-lv60-self": 9,
    "facebook/wav2vec2-large-slavic-voxpopuli-v2": 11,
    "classla/wav2vec2-large-slavic-parlaspeech-hr": 7,  
}
import os
# os.system("rm -r models/*")
from utils import train_model, eval_model

checkpoint = "/home/peterr/macocu/task14/models/classla_wav2vec2-large-slavic-parlaspeech-hr_best_model_finding_2_/classla_wav2vec2-large-slavic-parlaspeech-hr_best_model_finding_2_/checkpoint-1225/"

example_eval_config = {
    "output_column": "target",
    "model_name_or_path":  checkpoint,
    "eval_file": "014_PSs.csv"
}

y_true, y_pred = eval_model(example_eval_config)

pd.Series(y_pred).value_counts()

Using custom data configuration default-59eccc379fda4693


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /home/peterr/.cache/huggingface/datasets/csv/default-59eccc379fda4693/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


0ex [00:00, ?ex/s]

  0%|          | 0/1000 [00:00<?, ?ba/s]

neutral    1955
anger        43
sadness       2
dtype: int64

In [21]:
df = pd.read_csv("PSsample/PSsample.csv")
df["path"] = df.path.apply(lambda s: "PSsample/"+s)

In [22]:
results = pd.DataFrame(data={"path": [str(p) for p in paths],
                   "target": ["sadness" for p in paths],
                   "y_pred": y_pred}
             )

df = pd.merge(
      right = df,
      left = results,
      on = "path",
      how="left"
)
gb = df.groupby("Speaker_name").agg({
      "y_pred": lambda l: pd.Series(l).value_counts().to_dict()
})
gb

,y_pred
Speaker_name,
"Bačić, Branko","{'neutral': 198, 'anger': 2}"
"Bulj, Miro","{'neutral': 191, 'anger': 9}"
"Bunjac, Branimir","{'neutral': 194, 'anger': 5, 'sadness': 1}"
"Lovrinović, Ivan","{'neutral': 194, 'anger': 6}"
"Maras, Gordan","{'neutral': 195, 'anger': 5}"
"Mrak-Taritaš, Anka","{'neutral': 198, 'sadness': 1, 'anger': 1}"
"Panenić, Tomislav",{'neutral': 200}
"Pernar, Ivan","{'neutral': 192, 'anger': 8}"
"Čuraj, Stjepan","{'neutral': 194, 'anger': 6}"


In [23]:
df["path"] = df.path.apply(lambda s: s[9:])
df.head()

,path,target,y_pred,Unnamed: 0,orig_file,start,end,words,word_start_times,norm_words,...,split,norm_words_edited,Speaker_role,Speaker_type,Speaker_party,Speaker_party_name,Party_status,Speaker_name,Speaker_gender,Speaker_birth
0,ROfl1iWhWyw_12050.25-12056.19.wav,sadness,neutral,240235,"23 10 2019 - 1. dio, 14. sjednica, 9. saziv [R...",12050.25,12056.19,"['koja', 'ima', 'nekih', '369', 'EUR-a', 'po',...","[0, 0.19, 0.33, 0.81, 2.41, 2.7, 2.83, 3.28000...","['koja', 'ima', 'nekih', 'tristo', 'šezdeset',...",...,train,NaN,Regular,MP,HNS,Klub Hrvatske narodne stranke - liberalnih dem...,Coalition,"Čuraj, Stjepan",M,1980
1,UPHLmUzadcg_7063.96-7082.57.wav,sadness,neutral,228568,"1 3 2018 - 1. dio, 7. sjednica, 9. saziv [UPHL...",7063.96,7082.57,"['vi', 'morate', 'to', 'prijaviti', 'naravno,'...","[0, 0.96, 1.4100000000000001, 1.58, 2.02, 2.36...","['vi', 'morate', 'to', 'prijaviti', 'naravno',...",...,train,NaN,Regular,MP,Nezavisni,Klub nezavisnih,Opposition,"Pernar, Ivan",M,1985
2,LiGw8Wnne9c_8047.72-8067.17.wav,sadness,neutral,288441,"5 2 2020 - 2. dio, 16. sjednica, 9. saziv [LiG...",8047.72,8067.17,"['zaustaviti', 'hitno', 'neracionalnost,', 'ra...","[0, 0.7000000000000001, 1.5, 2.49, 3.14, 4.37,...","['zaustaviti', 'hitno', 'neracionalnost', 'ras...",...,train,NaN,Regular,MP,Živi zid i PH,Klub zastupnika Živog zida i Promijenimo Hrvatsku,Opposition,"Lovrinović, Ivan",M,1960
3,LZETjGQNaNk_20735.4-20755.38.wav,sadness,neutral,344642,"28 6 2018 - 2. dio, 8. sjednica, 9. saziv [LZE...",20735.40,20755.38,"['dao', 'taj', 'većinski', 'paket', 'i', 'otiš...","[0, 0.30000000000000004, 0.49, 0.99, 1.42, 1.4...","['dao', 'taj', 'većinski', 'paket', 'i', 'otiš...",...,train,NaN,Regular,MP,Most,Klub Mosta nezavisnih lista,Opposition,"Bulj, Miro",M,1972
4,lLuRksi0rvY_5215.06-5231.45.wav,sadness,neutral,257248,"27 6 2018 - 1. dio, 8. sjednica, 9. saziv [lLu...",5215.06,5231.45,"['o', 'tim', 'krajevima', 'nego', 'sada', 'ovi...","[0, 0.13, 0.38, 1.33, 1.58, 1.81, 1.9500000000...","['o', 'tim', 'krajevima', 'nego', 'sada', 'ovi...",...,train,NaN,Regular,MP,Most,Klub Mosta nezavisnih lista,Opposition,"Bulj, Miro",M,1972


In [24]:
df[["path", "y_pred"]].to_csv("PSsample/results.csv", index=False)